In [72]:
import pandas as pd
import numpy as np

In [73]:
# Imports CSVs from raw for merging
election_results_df = pd.read_csv('../processed_data/1988_to_2019_results_clean.csv')
gb_polls = pd.read_csv('../processed_data/gb_polls.csv')

In [74]:
# Fxes election year column name
election_results_df['election_year'] = election_results_df['Year']

In [75]:
# Fixes electon year for data type
gb_polls['election_year'] = gb_polls['next_elec_date'].str[:4]

In [76]:
# Fixes electon year for data type
gb_polls['election_year'] = gb_polls['election_year'].astype(int)

In [77]:
# Option 1: filters election year to pre 2019
# filtered_gb_polls_df = gb_polls[gb_polls['election_year'] <= 2020]

In [78]:
# Option 2: filters election year to post 2019
# filtered_gb_polls_df = gb_polls[gb_polls['election_year'] >= 2020]

In [79]:
# Option 3 merges dataframe fully
gb_polls_actuals = gb_polls.merge(election_results_df, on='election_year', how='left')

In [80]:
# Dictionary for column names
rename_columns = {
    'BRX': 'BRX_FC',
    'CON': 'CON_FC',
    'GRE': 'GRE_FC',
    'LAB': 'LAB_FC',
    'LIB': 'LIB_FC',
    'OTH': 'OTH_FC',
    'PLC': 'PLC_FC',
    'SNP': 'SNP_FC',
    'UKI': 'UKI_FC',
    'BRX_ACTUAL_PERCENTAGE': 'BRX_ACT',
    'CON_ACTUAL_PERCENTAGE': 'CON_ACT',
    'GRE_ACTUAL_PERCENTAGE': 'GRE_ACT',
    'LIB_ACTUAL_PERCENTAGE': 'LIB_ACT',
    'LABOUR_ACTUAL_PERCENTAGE': 'LAB_ACT',
    'PLC_ACTUAL_PERCENTAGE': 'PLC_ACT',
    'SNP_ACTUAL_PERCENTAGE': 'SNP_ACT',
    'UKI_ACTUAL_PERCENTAGE': 'UKI_ACT',
    'OTH_PERCENTAGE': 'OTH_ACT'
}

In [81]:
# Renames columns for uniformity
gb_polls_actuals.rename(columns=rename_columns, inplace=True)

In [82]:
# Drops irrelevant columns
gb_polls_actuals.drop(columns=['Country'], inplace=True)

In [83]:
# Renames df for sanity
combined_df = gb_polls_actuals

In [84]:
# Renames columns for space
combined_df.rename(columns={'pollster_rating': 'rating'}, inplace=True)
combined_df.rename(columns={'next_election_date': 'next_elec_date'}, inplace=True)
combined_df.rename(columns={'days_until_next_election': 'days_to_elec'}, inplace=True)

In [85]:
# Drops further columns
combined_df.drop(columns='Geography', inplace=True)  # Drop column


In [86]:
# Add a field 'poll length' that shows number of days the poll was held for
combined_df['poll_length'] = pd.to_datetime(combined_df.enddate) - pd.to_datetime(combined_df.startdate)
combined_df['poll_length'] = combined_df['poll_length'].dt.days

In [87]:
# Apply function to correct negative values and replace 0 with 1
def adjust_poll_length(days):
    if days < 0:
        return 0
    elif days == 0:
        return 1
    else:
        return days

In [88]:
combined_df['poll_length'] = combined_df['poll_length'].apply(adjust_poll_length)

In [89]:
# Converts start/end to datetime
combined_df['startdate'] = pd.to_datetime(combined_df['startdate'])
combined_df['enddate'] = pd.to_datetime(combined_df['enddate'])
combined_df['next_elec_date'] = pd.to_datetime(combined_df['next_elec_date'])

In [90]:
# Divide forecasts by 100 to create values between 0-1
for column in ['BRX_FC', 'CON_FC', 'GRE_FC', 'LAB_FC', 'LIB_FC', 'OTH_FC', 'PLC_FC', 'SNP_FC', 'UKI_FC']:
    combined_df[column] = combined_df[column] / 100

In [91]:
# Divide actuals by 100 to create values between 0-1
for column in ['BRX_ACT', 'CON_ACT', 'GRE_ACT', 'LIB_ACT', 'LAB_ACT', 'PLC_ACT', 'SNP_ACT', 'UKI_ACT', 'OTH_ACT']:
    combined_df[column] = combined_df[column] / 100

In [92]:
# Calculates month cleaning column
def calculate_month_diff(d1, d2):
    return (d2.year - d1.year) * 12 + d2.month - d1.month

In [93]:
# Applies month cleaning
combined_df['months_to_elec'] = combined_df.apply(lambda row: calculate_month_diff(row['enddate'], row['next_elec_date']), axis=1)

In [94]:
# Function to calculate weight
def calculate_weight(months, max_months=60):
    return max(0, 1 - np.log1p(months) / np.log1p(max_months))

In [95]:
# Applies weight calculation
combined_df['months_to_elec_weight'] = combined_df['months_to_elec'].apply(calculate_weight)

In [96]:
# Defines ruling periods
party_in_power = {
    '1985-01-01': 'Conservative',
    '1987-06-11': 'Conservative',
    '1992-04-09': 'Conservative',
    '1997-05-01': 'Labour',
    '2001-06-07': 'Labour',
    '2004-01-01': 'Labour',
    '2005-05-05': 'Labour',
    '2010-05-06': 'Conservative_Liberal',
    '2015-05-07': 'Conservative',
    '2017-06-08': 'Conservative',
    '2019-12-12': 'Conservative'
}

In [97]:
# Function to get party in power at a given date
def get_party_in_power(date):
    date = pd.to_datetime(date)
    for key in sorted(party_in_power.keys(), reverse=True):
        if date >= pd.to_datetime(key):
            return party_in_power[key]
    return None

In [98]:
# Applies party in power function
combined_df['party_in_power'] = combined_df['startdate'].apply(get_party_in_power)


In [99]:
combined_df.columns

Index(['Unnamed: 0_x', 'startdate', 'enddate', 'pollster', 'samplesize',
       'rating', 'next_elec_date', 'days_to_elec', 'BRX_FC', 'CON_FC',
       'GRE_FC', 'LAB_FC', 'LIB_FC', 'OTH_FC', 'PLC_FC', 'SNP_FC', 'UKI_FC',
       'election_year', 'Unnamed: 0_y', 'Year', 'BRX_ACT', 'CON_ACT',
       'GRE_ACT', 'LIB_ACT', 'LAB_ACT', 'PLC_ACT', 'SNP_ACT', 'UKI_ACT',
       'OTH_ACT', 'poll_length', 'months_to_elec', 'months_to_elec_weight',
       'party_in_power'],
      dtype='object')

In [100]:
order = ['startdate', 'enddate', 'pollster', 'samplesize', 'rating',
       'next_elec_date', 'days_to_elec', 'months_to_elec', 'months_to_elec_weight', 'poll_length',
       'party_in_power', 'CON_FC', 'LAB_FC', 'LIB_FC',
       'BRX_FC', 'GRE_FC', 'OTH_FC', 'PLC_FC', 'SNP_FC', 'UKI_FC', 'CON_ACT', 'LAB_ACT', 'LIB_ACT', 'BRX_ACT', 'GRE_ACT',
       'PLC_ACT', 'SNP_ACT', 'UKI_ACT', 'OTH_ACT']

In [101]:
combined_df = combined_df[order]

In [102]:
combined_df.columns

Index(['startdate', 'enddate', 'pollster', 'samplesize', 'rating',
       'next_elec_date', 'days_to_elec', 'months_to_elec',
       'months_to_elec_weight', 'poll_length', 'party_in_power', 'CON_FC',
       'LAB_FC', 'LIB_FC', 'BRX_FC', 'GRE_FC', 'OTH_FC', 'PLC_FC', 'SNP_FC',
       'UKI_FC', 'CON_ACT', 'LAB_ACT', 'LIB_ACT', 'BRX_ACT', 'GRE_ACT',
       'PLC_ACT', 'SNP_ACT', 'UKI_ACT', 'OTH_ACT'],
      dtype='object')

In [103]:
#Option 1: Saves to CSV for pre 2019
combined_df.to_csv('../processed_data/1988_to_2024_combined_clean_polling_and_results.csv', index=True)

In [ ]:
#Option 2: Saves to CSV for post 2019
# combined_df.to_csv('../processed_data/2019_to_2024_combined_clean_polling_and_results.csv', index=False)